In [ ]:
# import pyreadr
from glob import glob
import json
import pandas as pd
from itertools import product
import numpy as np
from scipy.special import ndtri, ndtr
from scipy.optimize import least_squares

In [ ]:
with open("cems.json", 'r') as fh:
    data = json.load(fh)

In [ ]:
preferences = data['preferences']
df = pd.DataFrame(preferences)
df = df[~(df == 'NA').apply(lambda row: np.any(row), axis=1)]

In [ ]:
df.school1.unique()

In [ ]:
df.school2.unique()

In [ ]:
df["pair"] = df.apply(lambda row: (row['school1'], row['school2']), axis=1)

gb = df.groupby("pair")

tbl1 = gb[['win1', 'tied', 'win2']].sum()
tbl1

In [ ]:
data = (tbl1 + tbl1[['tied']].values/2).loc[:, ('win1', 'win2')]
data

In [ ]:
# data.apply(lambda x, y: x+y, axis=1)
# data.win1 / (data.win1 + data.win2)
# ['Stockholm', 'Barcelona

p = dict((data.win1 / (data.win1 + data.win2)))
p

In [ ]:
school2idx = {y: x for x, y in enumerate(set(y for x in data.index for y in x))}

A = np.zeros((len(school2idx), len(school2idx)))
for s1, s2 in data.index:
    A[school2idx[s1], school2idx[s2]] += 1
schools_sorted = [list(school2idx)[idx] for idx in np.argsort(A.sum(axis=0))]
rev_schools_sorted = list(reversed(schools_sorted))
# list(reversed(schools_sorted))
schools_sorted

In [ ]:
def cost(mu):
    mu_1 = {schools_sorted[i]: mu[i] for i in range(len(mu))}
    mu_1['Stockholm'] = 0
    return np.array([(v - ndtr(mu_1[s1] - mu_1[s2]))**2 for (s1, s2), v in p.items()])

In [ ]:
np.sum(cost(np.zeros(5))), np.sum(cost(np.ones(5)))

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def logit(y):
    return -np.log((1 - y) / y)

def compute_mu(data, F=ndtri):
    mu = {}
    for (s1, s2), p in (data.win1 / (data.win1 + data.win2)).items():
        if s2 == 'Stockholm':
            mu[s1] = F(p)
    return mu

def to_np(d):
    return np.array([d[city] for city in schools_sorted[:-1]])

In [ ]:
np.sum(cost(to_np(compute_mu(data))))

In [ ]:
mu = {city: [] for city in school2idx}
mu['Stockholm'] = [0]

for city in rev_schools_sorted:
    table = p.copy()
    for (s1, s2), value in table.items():
        if s2 == city:
            # p_ij = F(mu_i - mu_j)
            # mu_i = Fi(p_ij) + mu_j
            mu[s1].append(ndtri(value) + mu[city][-1])
#             mu[s1].append(value)

In [ ]:
mu

In [ ]:
mu_1 = {l: np.mean(m) for l, m in mu.items()}
# mu
# ndtri(0.7511551155)
mu_1

In [ ]:
np.sum(cost(to_np(mu_1)))

In [ ]:
optimized_mu = least_squares(cost, to_np(mu_1)).x
# optimized_mu = least_squares(cost, np.zeros(5)).x
# least_squares(cost, np.zeros(5))
# least_squares(cost, to_np(mu_1))

In [ ]:
{city: optimized_mu[i] for i, city in enumerate(schools_sorted[:-1])}

In [ ]:
import jax
import jax.numpy as np

In [ ]:
# least_squares(cost, to_np(mu_1), jac=jax.jacfwd(jax.grad(cost)))

In [ ]:
[(v - ndtr(mu_1[s1] - mu_1[s2]))**2 for (s1, s2), v in p.items()]

In [ ]:
# cost(np.array(mu_1.values()))
cost(np.array(list(mu_1.values())))
np.array(list(mu_1.values()))

In [ ]:
mu_1

In [ ]:
school2idx

In [ ]:
mu = compute_mu(data, ndtri)
mu

In [ ]:
logit(0.635314)
0.33 / 0.34
0.99 / 1.064

In [ ]:
mu = pd.Series(mu)

In [ ]:
mu / sum(mu)

In [ ]:
ndtr(0.982 - 0.561)
ndtr(mu['London'] - mu['Paris'])